In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [546]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [ ]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [245]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50

In [248]:


data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2)
newarr=jnp.zeros((int(orderbookLevels*2),7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 7)


In [622]:
horizon=500

orderbook_1=JaxOrderBook(price_levels=50,orderQueueLen=50)
trades_init=orderbook_1.process_orders_array(initOB)
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon])

In [623]:
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1]).astype(float)
#print(L2OB.reshape(50,4))
print((myOB.reshape(50,4)-L2OB.reshape(50,4)).astype(int))#L2OB#-myOB

[[       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0      

In [619]:
ordersJNP[horizon-1+8]

Array([1.000000e+00, 1.000000e+00, 5.000000e+01, 2.200000e+06,
       0.000000e+00, 1.256694e+06, 3.420910e+04], dtype=float32)

In [624]:
orderbook_1.orderbook_array[1,39,:,:]

Array([[ 9.7000000e+01,  2.2000000e+06,  0.0000000e+00, -9.9999900e+05,
         3.4200000e+04],
       [ 2.5000000e+02,  2.2000000e+06,  0.0000000e+00,  7.8690600e+06,
         3.4203176e+04],
       [ 1.8000000e+01,  2.2000000e+06,  0.0000000e+00,  3.0572000e+04,
         3.4209102e+04],
       [ 1.0000000e+01,  2.2000000e+06,  0.0000000e+00,  3.0591000e+04,
         3.4209102e+04],
       [ 5.0000000e+01,  2.2000000e+06,  0.0000000e+00,  3.0608000e+04,
         3.4209102e+04],
       [ 7.0000000e+00,  2.2000000e+06,  0.0000000e+00,  3.0654000e+04,
         3.4209102e+04],
       [ 4.5000000e+01,  2.2000000e+06,  0.0000000e+00,  3.0677000e+04,
         3.4209102e+04],
       [ 3.0000000e+00,  2.2000000e+06,  0.0000000e+00,  3.0681000e+04,
         3.4209102e+04],
       [ 3.0000000e+01,  2.2000000e+06,  0.0000000e+00,  3.0693000e+04,
         3.4209102e+04],
       [ 2.0000000e+00,  2.2000000e+06,  0.0000000e+00,  8.2866000e+04,
         3.4209102e+04],
       [ 1.3000000e+01,  2.200

## Load the original orderbook implementation on the CPU

In [398]:
orderbook_2=cpuOrderBook()

In [399]:
limit_orders = []
order_id_list = [15000000 + i for i in range(20)]
for i in range(10):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],
          "timestamp": timestamp}
    limit_orders.append(item)
    
for i in range(10):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i+10],
          "timestamp": timestamp}
    limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

In [405]:
##Taken largely from the Example.py file in the orderbook repository for the cpu

horizon=2
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype = l1[1] 
    side = 'bid' if l1[5] ==1 else 'ask'
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    
    
    print(message)
    orderbook_2.processOrder(message,True,False)

        
    

{'type': 5, 'side': 'bid', 'quantity': 13, 'price': 2228300, 'trade_id': 0, 'timestamp': '34200.00678936', 'order_id': 0}
{'type': 1, 'side': 'bid', 'quantity': 100, 'price': 1870200, 'trade_id': 7381730, 'timestamp': '34200.030569044', 'order_id': 7381730}


In [406]:
print(orderbook_2)

====================================== Asks ======================================
Quantity    40  |  Price  2257500  |  Order_ID   15000009  |  Time 34200.000000001
Quantity    20  |  Price  2252500  |  Order_ID   15000008  |  Time 34200.000000001
Quantity   100  |  Price  2250000  |  Order_ID   15000007  |  Time 34200.000000001
Quantity    20  |  Price  2247500  |  Order_ID   15000006  |  Time 34200.000000001
Quantity    20  |  Price  2242500  |  Order_ID   15000005  |  Time 34200.000000001
Quantity   200  |  Price  2240000  |  Order_ID   15000004  |  Time 34200.000000001
Quantity   100  |  Price  2237500  |  Order_ID   15000003  |  Time 34200.000000001
Quantity   300  |  Price  2237200  |  Order_ID   15000002  |  Time 34200.000000001
Quantity    50  |  Price  2234000  |  Order_ID   15000001  |  Time 34200.000000001
Quantity  1507  |  Price  2229100  |  Order_ID   15000000  |  Time 34200.000000001


Quantity   300  |  Price  2226400  |  Order_ID   15000010  |  Time 34200.000000001
Qu